* Verify Flux closure on circuit loops
* Evaluate states of circuits
* Calculate expectation of node flux
* Evaluate quantization condition

This seems to be a counter-logical idea to solve the circuits.
Since, the flxuoid variables are defined on the nodes, and the branch flux is derived from them
Therefore, the rederivation of closure condition is trivially true.

In [66]:
from DiSuQ.Torch import models
from DiSuQ.Torch.circuit import wavefunction
from DiSuQ.Torch.components import flux_quanta
from DiSuQ.Torch.dense import chargeStates
from torch import tensor
from numpy import arange,linspace,meshgrid,array,around,sqrt
from DiSuQ.utils import plotCompare,plotHeatmap
from torch import set_num_threads
set_num_threads(32)

In [47]:
flux_range = tensor(linspace(0,1,3))
flux_profile = [{'I':flux} for flux in flux_range]
print(flux_profile)

[{'I': tensor(0., dtype=torch.float64)}, {'I': tensor(0.5000, dtype=torch.float64)}, {'I': tensor(1., dtype=torch.float64)}]


### charge basis

In [3]:
basis = [8,8,8]; rep = 'Q'
circuit = models.shuntedQubit(basis,sparse=False,symmetry=True)
print(circuit.circuitComponents())

{'JJ1': 120.0, 'C1': 0.049999989569187164, 'JJ2': 50.0, 'C2': 0.009999998845160007, 'JJ3': 120.0, 'C3': 0.049999989569187164, 'I': 0.0002533029764890671}


In [4]:
H_LC = circuit.chargeHamiltonianLC()
H_J = circuit.josephsonCharge

In [5]:
psi0,psi1 = wavefunction(H_LC+H_J(flux_profile[1]),[0,1])

In [62]:
psi0.conj()@psi0, psi1.conj()@psi1, psi0.conj()@psi1

(tensor(1.0000+0.j, grad_fn=<DotBackward0>),
 tensor(1.0000+0.j, grad_fn=<DotBackward0>),
 tensor(6.2049e-08+1.5898e-13j, grad_fn=<DotBackward0>))

In [63]:
plotCompare(arange(len(psi0)),{'psi0':psi0.abs().detach().numpy(),'psi1':psi1.abs().detach().numpy()})

In [51]:
circuit.modeImpedance()

[tensor(0.1883, grad_fn=<SqrtBackward0>),
 tensor(inf, grad_fn=<SqrtBackward0>),
 tensor(inf, grad_fn=<SqrtBackward0>)]

In [76]:
flux = [circuit.operatorExpectation(psi0,circuit.backend.basisFq,node,psi0).real.detach().item() for node in range(3)]

In [72]:
charge = [circuit.operatorExpectation(psi1,circuit.backend.basisQq,node,psi1).real.detach().item() for node in range(3)]

In [73]:
sqrt(flux_quanta)

4.5473440954062344e-08

In [77]:
flux = flux/sqrt(flux_quanta)

In [81]:
flux[0]-0,flux[1]-flux[0],flux[2]-flux[1],0-flux[2]

(0.03972804709740187,
 0.07055113618481125,
 0.9729483623044123,
 -1.0832275455866254)

In [75]:
flux[0]-0,flux[1]-flux[0],flux[2]-flux[1],flux[2]-0

(0.07211441553639725, 0.23533965147278146, 0.3074540670091787)

In [57]:
charge

[0.0001429461408406496, -0.00014808494597673416, -8.585042087361217e-05]

### kerman basis

In [ ]:
basis = [1,1,1]; rep = 'Q'
basis = [10,10]
basis = {'O':[4],'J':[8,8],'I':[]}; rep = 'K'
circuit = models.shuntedQubit(basis,cap=[C11,C12,C22],sparse=False)
print(circuit.circuitComponents())

In [ ]:
H_LC = circuit.kermanHamiltonianLC()
H_J = circuit.kermanHamiltonianJosephson